In [ ]:
"""Decoy generation workflow"""

# Import relevant packages
from CGRtools.files import RDFRead, RDFWrite
from CGRtools.exceptions import *
# from ..util.utils import (generate_reactions, remove_reagents,
#                           containers_split, not_radical,
#                           get_rules)
# from ..util.routine import (_save_log, _util_file)
from datetime import date

import os
import math
import multiprocessing
import pickle
import time
import argparse


def main(n_proc):
    """
    Main generation routine

    =======================   ===================================
    Reactions in input file   1 316 541
    Data                      reactions, type: ReactionContainer
    =======================   ===================================

    Parameters
    ----------
    :param n_proc: Num pool worker

    Returns
    -------
    :return: None
    """

    with open("Config.pickle", "rb") as configFile:
        config_list = pickle.load(configFile)
        name_in, templates_fn, name_out, log_filename, batch, max_decoys, limit, v, log = config_list

    with open(templates_fn, "rb") as pkl:
        templates = pickle.load(pkl)

    with RDFRead(name_in, indexable=True) as data, \
    open(name_out, "a") as w, RDFWrite(w) as rdf, \
    open("NonRecon_smiles.rdf", "a") as z, RDFWrite(z) as nrdf, \
    open("Removed_smiles.rdf", "a") as y, RDFWrite(y) as rmvd:

        name = "Worker-{}".format(n_proc)
        id_start = n_proc * batch
        id_stop = id_start + batch

        # LOGGING
        if log:
            _save_log(log_filename, str("Process {} initialized\n".format(name)))
        start_time = time.time()

        for reaction in data[id_start: id_stop]:
            doc = {}
            reaction = remove_reagents(reaction)
            if reaction is not None:
                reaction = containers_split(reaction)
                if len(reaction.reactants) == 2:
                    cgr = reaction.compose()
                    if not_radical(cgr):
                        if db_check(cgr): # the num of dyn bonds in a CGR must be > 1 and < or = 7
                            reaction.meta.update({"type": "Initial"})
                            doc.update({str(reaction): {"structure": reaction,
                                                        "type": reaction.meta["type"]}})

                            doc = generate_reactions(reaction, reaction.reactants, templates,
                                                     max_decoys, limit, doc)

                            if any([True if inner_dict["type"] == "Reconstructed" else False for inner_dict in
                                    doc.values()]):
                                if v:
                                    print("REC;ID:{};NUM:{}\n".format(
                                            reaction.meta["Reaction_ID"], len(doc)))
                                if log:
                                    _save_log(log_filename,
                                              str("REC;ID:{};NUM:{}\n".format(
                                                  reaction.meta["Reaction_ID"], len(doc))))
                                for rxn in [rxn["structure"] for rxn in doc.values()]:
                                    rdf.write(rxn)
                            else:
                                if v:
                                    print("NREC;ID:{};NUM:{}\n".format(reaction.meta["Reaction_ID"], len(doc)))
                                if log:
                                    _save_log(log_filename,
                                              str("NREC;ID:{};NUM:{}\n".format(
                                                  reaction.meta["Reaction_ID"], len(doc))))
                                for rxn in [rxn["structure"] for rxn in doc.values()]:
                                    nrdf.write(rxn)
                        else:
                            reaction.meta.update({"removed": "dynbonds"})
                            rmvd.write(reaction)
                    else:
                        reaction.meta.update({"removed": "radical"})
                        rmvd.write(reaction)
                else:
                    reaction.meta.update({"removed": "nonbinary"})
                    rmvd.write(reaction)
        else:
            end_time = time.time()
            if v:
                print("Process {} finished batch processing in time: {}s\n".format(name, end_time - start_time))
            if log:
                _save_log(log_filename,
                          str("Process {} finished batch processing in time: {}s\n".format(name,
                                                                                           end_time - start_time)))
    return


if __name__ == '__main__':
    name_in = "/home/vendinm/caspFilter/data/decoyGeneration/test_folder/TEST.rdf"
    template_pkl = "/home/vendinm/caspFilter/data/templateGeneration/caspRules2022-02-26.pickle"
    num_proc = 25
    log_filename = "GENERATE_DECOYS_LOG_SMILES.txt" # smth
    name_out = 'test_smiles_{}.rdf'.format(date.today())
    batch = 10000
    v = False
    num = 50
    lim = 10
    log = True
    
    _util_file("NonRecon_smiles.rdf")  # deleting the rdf files if it was created earlier in the directory
    _util_file("Removed_smiles.rdf")
    _util_file(name_out)
    if log: 
        _util_file(log_filename)
    
    with open("Config.pickle", "wb") as config:
        config_list = [
            str(name_in),
            str(template_pkl),
            str(name_out),
            str(log_filename),
            int(batch),
            int(num),
            int(lim),
            bool(v),
            bool(log)
        ]
        pickle.dump(config_list, config)

    with RDFRead(name_in, indexable=True) as file:
        chunk = math.ceil(len(file) // 10000)
        
    with multiprocessing.Pool(processes=num_proc) as pool:
        pool.map(main, list(range(chunk)))
        

In [3]:
"""Some routines for generation workflow"""
# from routine import (_remove_mols, _db_check)
from CGRtools.reactor import Reactor
from CGRtools.containers import ReactionContainer
from CGRtools.exceptions import (InvalidAromaticRing,
                                 MappingError)


def remove_reagents(reaction):
    """
    Removing unchanging molecules, and checking reaction properties
    :param reaction: input reaction
    :return: ReactionContainer or None in some cases
    """
    try:
        cgr = reaction.compose()
    except ValueError:
        return None
    
    to_remove = []
    for n, i in enumerate(cgr.split()): # remove unchanging molecules
        if any([z[1].charge != z[1].p_charge for z in i.atoms()]) or \
           any([z[2].order != z[2].p_order for z in i.bonds()]):
            continue
        to_remove.extend([x for x, _ in i.atoms()])
    for mol in reaction.molecules():
        for z in to_remove:
            try:
                mol.delete_atom(z)
            except KeyError:
                continue

    try:
        reaction.canonicalize()
    except InvalidAromaticRing:
        return None
    reaction.flush_cache()
    reactants = [x for x in reaction.reactants if x]
    products = [x for x in reaction.products if x]
    if len(reactants) > 0 and len(products) > 0:
        reaction = ReactionContainer(reactants=reactants,
                                     products=products,
                                     meta=reaction.meta)
        return reaction
    else:
        return None


def db_check(cgr):
    if len(cgr.center_bonds) > 1 and len(cgr.center_bonds) <= 7:
        return True
    else:
        return False


def not_radical(cgr):
    """
    Checking for charged atoms in a Condensed Graph of Reaction
    :param cgr: Condensed Graph of the input reaction
    :return: bool
    """
    if cgr and cgr.center_atoms:
        if any(x.is_radical or x.p_is_radical for _, x in cgr.atoms()):
            return False
    return True


def containers_split(reaction):
    """
    Separation of MoleculeContainers
    :param reaction: input reaction
    :return: ReactionContainer
    """
    new_reactants = [x for container in reaction.reactants for x in container.split()]
    new_products = [x for container in reaction.products for x in container.split()]
    return ReactionContainer(reactants=new_reactants,
                             products=new_products,
                             meta=reaction.meta)


def generate_reactions(reaction, reactants, templates, max_decoys, limit, doc):
    """
    Accumulation of generated reactions
    :param reaction: input reaction
    :param reactants: reactants of input reaction
    :param templates: template list of reaction transformations (list[ReactionContainer, ...])
    :param max_decoys: max number of reaction to generate
    :param limit: max number of reaction from one transformation
    :param doc: Dict[..., Dict[...]}]
    """
    for rxn in apply_templates(reaction, reactants, templates, limit):
        if len(doc) == max_decoys:
            break

        new_reaction = ReactionContainer(reactants=reactants,
                                         products=rxn.products,
                                         meta=rxn.meta)
        new_reaction.meta.update(reaction.meta)
        try:
            new_reaction = containers_split(new_reaction)
            new_reaction.canonicalize()
        except InvalidAromaticRing:
            continue
        try:
            if (new_reaction.compose()).center_atoms:
                try:
                    if str(new_reaction) not in doc:
                        new_reaction.meta.update({"type": "Decoy"})
                    elif str(new_reaction) in doc and \
                        doc[str(new_reaction)]["type"] == "Initial":
                        new_reaction.meta.update({"type": "Reconstructed"})
                    else:
                        continue
                except KeyError:
                    continue

                doc.update({str(new_reaction): {"structure": new_reaction,
                                                "type": new_reaction.meta["type"]}})
            else:
                continue
        except MappingError:
            continue
    return doc


def apply_templates(reaction, reactants, templates, limit):
    """
    New reaction generator
    :param reactants: reactants of input reaction
    :param templates: template list of reaction transformations (list[ReactionContainer, ...])
    :param limit: max number of reaction from one transformation
    :return: yield(ReactionContainer) of generated reaction
    """
    strict_templates = get_rules(reaction)
    
    for template in strict_templates:
        try:
            templates.remove(template)
        except ValueError:
            pass
        templates.insert(0, template)
    
    reactors = [Reactor(template,
                        delete_atoms=True,
                        one_shot=True,
                        automorphism_filter=False) for template in templates]  # NB! CGRtools v. > 4.1.22
    queue = [r(reactants) for r in reactors]
    seen = set()
    while queue:
        reactor_call = queue.pop(0)
        try:
            num = 0
            for new_reaction in reactor_call:
                if num + 1 == limit:
                    break
                if str(new_reaction) not in seen:
                    seen.add(str(new_reaction))
                    num += 1
                    yield new_reaction
        except KeyError:
            continue
            
            
def get_strict_templates(rxn_id, templates):
    template_set = []
    for template in templates:
        if rxn_id in template.meta["id_in_rdf"]:
            template_set.append(template)
    return template_set


def get_rules(reaction):
    """
    Obtaining the rules of reaction transformations
    NB! CGRtools.enumerate_centers() is used
    :param reaction: input reaction
    :return: list[ReactionContainer, ...]
    """
    reaction.clean_stereo()
    rules = []

    for n, partial_reaction in enumerate(reaction.enumerate_centers()):  # getting single stages
        if n == 5:
            break  # without an endless loop may appear, cause of many reaction centers
        for reaction_center in partial_reaction.extended_centers_list:

            reactants = reaction.reactants
            products = reaction.products

            cleavage = set(partial_reaction.reactants).difference(partial_reaction.products)
            coming = set(partial_reaction.products).difference(partial_reaction.reactants)

            bare_reaction_center = set(partial_reaction.compose().center_atoms)
            
            rule_reac = [x.substructure(set(reaction_center).intersection(x),
                                        as_query=True) for x in reactants if set(reaction_center).intersection(x)]
            rule_prod = [x.substructure(set(reaction_center).intersection(x),
                                        as_query=True) for x in products if set(reaction_center).intersection(x)]

            if len(rule_reac) != 2:
                continue

            rule = ReactionContainer(reactants=rule_reac,
                                     products=rule_prod,
                                     meta=reaction.meta)

            for molecule in rule.molecules():
                molecule._rings_sizes = {x: () for x in molecule._rings_sizes}  # getting rid of the ring sizes info (NEC.)
                molecule._hydrogens = {x: () for x in molecule._hydrogens}  # getting rid of the hydrogen info (NEC.)

            for at_env in set(reaction_center).difference(cleavage.union(coming).union(bare_reaction_center)):

                for x in rule.reactants:
                    if at_env in x.atoms_numbers:
                        x._neighbors[at_env] = ()

                for x in rule.products:
                    if at_env in x.atoms_numbers:
                        x._neighbors[at_env] = ()

            for del_hyb_atom in cleavage:
                for molecule in rule.reactants:
                    if del_hyb_atom in molecule:
                        molecule._hybridization[del_hyb_atom] = ()  # getting rid of the hybridization info in react. (NEC.)

            for del_hyb_atom in coming:
                for molecule in rule.products:
                    if del_hyb_atom in molecule:
                        molecule._hybridization[del_hyb_atom] = ()  # getting rid of the hybridization info in prod. (NEC.)

            rule.flush_cache()
            rules.append(rule)
    return rules


In [4]:
"""Routine functions"""
from CGRtools.files import (RDFRead, RDFWrite)
from tqdm import tqdm

import pickle
import os


def _load_pkl(filename, n):
    with open("{}_{}.pickle".format(filename, n), "rb") as f:
        for i in range(2 ** 64):
            try:
                yield pickle.load(f)
            except EOFError:
                break


def _dump_pkl(filename, to_save, n):
    with open("{}_{}.pickle".format(filename, n), "ab") as f:
        pickle.dump(to_save, f)
    return


def _util_file(filename):
    try:
        os.remove(filename)
    except FileNotFoundError:
        pass


def _save_log(filename, string):
    with open(filename, "a") as flog:
        flog.write(string)


def RDFclean(RDFfilename, log, dump_size, dump_fn, v):
    """
    Dump pickling and removing duplicates
    :param RDFfilename: RDF file to check duplicates and examine
    :param log: log if necessary
    :param dump_size: size of dict to dump
    :param dump_fn: outer file name
    :param v: printing if necessary
    """
    with RDFRead(RDFfilename, indexable=True) as file:

        to_save = dict()
        log_filename = "CLEANING_LOG.txt"
        flag = True
        flag_pass = True

        for n, reaction in enumerate(tqdm(file), start=1):
            if n != 2210001 and flag_pass:
                continue
            else:
                flag_pass = False
            if flag:
                num = n - 1
                flag = False
            try:
                if str(reaction.compose()) not in to_save:
                    to_save.update({str(reaction.compose()): reaction})
                else:
                    if reaction.meta["type"].startswith("Reconstructed"):
                        if v: print("Replaced by reconstructed: {}".format(reaction.meta["Reaction_ID"]))
                        if log: _save_log(log_filename,
                                          str("Replaced by reconstructed: {}\n".format(reaction.meta["Reaction_ID"])))
                        del to_save[str(reaction.compose())]
                        to_save.update({str(reaction.compose()): reaction})
                    elif reaction.meta["type"].startswith("Decoy"):
                        if v: print("Founded duplicate decoy: {}, real id: {}".format(reaction.meta["Reaction_ID"],
                                                                                      to_save[
                                                                                          str(reaction.compose())].meta[
                                                                                          "Reaction_ID"]))
                        if log: _save_log(log_filename, str("Founded duplicate decoy: {}, real id: {}\n".format(
                            reaction.meta["Reaction_ID"],
                            to_save[str(reaction.compose())].meta["Reaction_ID"])))
                        continue
            except Exception as e:
                if v: print("{} was occurred, number: {}, rxn_ID: {}\n".format(e, n, reaction.meta["Reaction_ID"]))
                if log: _save_log(log_filename, str("{} was occurred, number: {}, rxn_ID: {}\n".format(e, n,
                                                                                                      reaction.meta[
                                                                                                          "Reaction_ID"]
                                                                                                       )))
                continue
            if n % dump_size == 0:
                flag = True
                try:
                    for rxn_dict in _load_pkl(dump_fn, num):
                        for duplicate in set(to_save).intersection(set(rxn_dict)):
                            try:
                                if to_save[duplicate].meta["type"].startswith("Reconstructed"):
                                    del rxn_dict[duplicate]
                                elif to_save[duplicate].meta["type"].startswith("Decoy"):
                                    del to_save[duplicate]
                            except KeyError:
                                del to_save[duplicate]
                                continue
                        _dump_pkl(dump_fn, rxn_dict, n)
                    else:
                        _dump_pkl(dump_fn, to_save, n)
                        to_save = dict()
                        _util_file("{}_{}.pickle".format(dump_fn, num))
                except (FileNotFoundError, UnboundLocalError):
                    _dump_pkl(dump_fn, to_save, n)
                    to_save = dict()
        else:
            for rxn_dict in _load_pkl(dump_fn, num):
                for duplicate in set(to_save).intersection(set(rxn_dict)):
                    try:
                        if to_save[duplicate].meta["type"].startswith("Reconstructed"):
                            del rxn_dict[duplicate]
                        elif to_save[duplicate].meta["type"].startswith("Decoy"):
                            del to_save[duplicate]
                    except KeyError:
                        del to_save[duplicate]
                        continue
                _dump_pkl(dump_fn, rxn_dict, n)
            else:
                _dump_pkl(dump_fn, to_save, n)
                _util_file("{}_{}.pickle".format(dump_fn, num))

def Compile(input_file, output_file):
    with open(input_file, "rb") as file, \
            open(output_file, "ab") as new_file, \
            open("Nonrecon.pickle", "ab") as Nonrec_file:
        counter = {}
        len_list = []
        nonrec_counter = {}
        num_examined = 0
        id_len = 0
        init = "Not found yet"

        for i in range(2 ** 64):
            try:
                data = pickle.load(file)
                len_list.append(len(data))
                if i == 0:
                    pass
                else:
                    id_len += 1
            except EOFError:
                break

            for n, reaction in enumerate(tqdm(data.values())):
                num_examined += 1
                try:
                    if reaction.meta["Reaction_ID"] != init:
                        try:
                            if counter[init]["Recon_str"] != 0:
                                pass
                            else:
                                nonrec_counter.update({init: counter[init]})
                                del counter[init]
                        except KeyError:
                            print("KeyError occurred: {}".format(reaction.meta["Reaction_ID"]))
                            pass

                        if num_examined >= len_list[id_len - 1]:
                            pickle.dump(counter, new_file)
                            pickle.dump(nonrec_counter, Nonrec_file)
                            num_examined = 0
                            counter = {}
                            nonrec_counter = {}

                        init = reaction.meta["Reaction_ID"]
                        num = 0
                        num_random = 0
                        num_strict = 0
                        structures = []
                        recon_str = 0
                        rand_ids = []
                    structures.append(reaction)
                    num += 1
                    if not reaction.meta["type"].startswith("Reconstructed"):
                        if "Rule_ID" in reaction.meta:
                            num_random += 1
                            rand_ids.append(reaction.meta["Rule_ID"])
                        else:
                            num_strict += 1
                    else:
                        recon_str = reaction
                    counter.update({reaction.meta["Reaction_ID"]:
                        {
                            "Recon_str": recon_str,
                            "Numbers": num,
                            "Structures": structures,
                            "Number of decoys from strict": num_strict,
                            "Number of decoys from random": num_random,
                            "Random rules ID's": rand_ids
                        }
                    })
                except Exception as e:
                    print("{} was occurred, number: {}".format(e, n))
                    continue
